In [1]:
import uproot
import pandas as pd
import ROOT

Welcome to JupyROOT 6.22/02


## Different Approaches:

---

**Approach - 1**

---

In this case, there are Downstream Dumper Data Generated and The Ghost Dumper, take the Downstream Tracks(NoVelo,No UT, Not an Electron) and merge it with Complete Ghost Dumper Data.
Classify them as the downstream and the ghost.
Ghost: The tracks that do not have a MC particle associated.

**Approach - 2***

---
In this case, split the data into DownStream and Ghost, train the model on Downstream, apply that model on Ghost Data, use this set as a test sample to evaluate the model performance.

In [2]:
DDvariables=['track_charge','track_chi2','track_chi2PerDoF','track_nLHCbIDs','track_p','track_phi','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity',
           'particle_hasScifi','particle_hasUT','particle_hasVelo','particle_isDown','particle_isDown_noVelo','particle_isLong',
           'particle_isLong_andUT','particle_isElectron']


GDvariables=['track_charge','track_chi2','track_chi2PerDoF','track_nLHCbIDs','track_p','track_phi','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity']



features=['track_chi2PerDoF','track_nLHCbIDs','track_p','track_position_x',
           'track_position_y','track_position_r','track_pt','track_tx','track_ty','track_pseudoRapidity']


tracks=['particle_hasScifi','particle_hasUT','particle_hasVelo','particle_isDown','particle_isDown_noVelo','particle_isLong',
           'particle_isLong_andUT','partiole_isElectron']

In [3]:
DDks=uproot.open('/data3/tracking-upgrade-data/dump_comb/DD_ks_moore_big.root:Particle_Data')
DDl=uproot.open('/data3/tracking-upgrade-data/dump_comb/DD_l_moore_big.root:Particle_Data')
GDks=uproot.open('/data3/tracking-upgrade-data/dump_comb/GD_ks_moore_big.root:Particle_Data')
GDl=uproot.open('/data3/tracking-upgrade-data/dump_comb/GD_l_moore_big.root:Particle_Data')

In [4]:
%%time

DDks_Data=DDks.arrays(DDvariables,library='pd')
DDl_Data=DDl.arrays(DDvariables,library='pd')


GDks_Data=GDks.arrays(GDvariables,library='pd')
GDl_Data=GDl.arrays(GDvariables,library='pd')



DD_data=pd.concat([DDks_Data,DDl_Data]).fillna(False)

GD_data=pd.concat([GDks_Data,GDl_Data]).fillna(False)


DD_data['Ghost']=0
GD_data['Ghost']=1

CPU times: user 3min 4s, sys: 8.23 s, total: 3min 12s
Wall time: 3min 11s


In [5]:
# DD_data=DD_data.loc[(DD_data['track_p']<100e3) & (DD_data['particle_isDown_noVelo']==True) | (DD_data['particle_isLong_andUT']==True) &(DD_data['particle_isElectron']==False)]


# GD_data=GD_data.loc[(GD_data['track_p']<100e3)]



# DD_data=DD_data.sample(5000)
# GD_data=GD_data.sample(5000)

In [6]:
compiled_data=pd.concat([DD_data,GD_data]).fillna(False)

In [7]:
#Approach -1


Downstream_Ghost=compiled_data.loc[((compiled_data['particle_isDown_noVelo']==True) & (compiled_data['particle_isElectron']==False) ) | (compiled_data['Ghost']==True)]

Downstream_Ghost.rename(columns={'Ghost':'Downstream/Ghost'},inplace=True)


Downstream_Ghost

/home/hashmi/.conda/envs/remote/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,track_charge,track_chi2,track_chi2PerDoF,track_nLHCbIDs,track_p,track_phi,track_position_x,track_position_y,track_position_r,track_pt,...,track_pseudoRapidity,particle_hasScifi,particle_hasUT,particle_hasVelo,particle_isDown,particle_isDown_noVelo,particle_isLong,particle_isLong_andUT,particle_isElectron,Downstream/Ghost
11,-1.0,11.941431,0.995119,12.0,14540.910267,2.857770,-178.011703,169.667587,245.917175,1158.375945,...,3.221504,True,True,False,True,True,False,False,False,0
22,-1.0,6.146245,0.512187,12.0,8404.460750,2.815944,232.126709,142.693268,272.477848,535.980539,...,3.444549,True,True,False,True,True,False,False,False,0
46,1.0,0.641681,0.053473,12.0,3977.934379,0.012414,780.580444,54.337784,782.469440,1165.989947,...,1.898133,True,True,False,True,True,False,False,False,0
47,1.0,0.742738,0.061895,12.0,2560.789084,0.241315,730.703308,773.573059,1064.115878,1053.322488,...,1.536248,True,True,False,True,True,False,False,False,0
61,1.0,16.254108,1.625411,10.0,1363.295797,0.015996,375.927368,642.561951,744.450970,813.778484,...,1.105036,True,True,False,True,True,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198347,1.0,6.500820,0.590984,11.0,2937.316736,-0.035631,1341.124146,-144.742264,1348.912264,1270.135015,...,1.481109,False,False,False,False,False,False,False,False,1
1198348,1.0,9.292344,0.929234,10.0,2523.196427,-0.021405,1438.449097,-158.623398,1447.168679,1234.037354,...,1.342371,False,False,False,False,False,False,False,False,1
1198349,1.0,16.462654,1.371888,12.0,1959.058243,-0.001669,1571.927002,-29.475683,1572.203331,1152.707891,...,1.122887,False,False,False,False,False,False,False,False,1
1198350,1.0,5.565911,0.505992,11.0,2960.389614,-0.068032,575.971497,-88.923218,582.795422,978.993853,...,1.771163,False,False,False,False,False,False,False,False,1


In [8]:

#save

file=uproot.recreate('./DataGenerated/01_Downsteam_Ghost.root')
file['Particle_Data']=Downstream_Ghost

In [9]:
#Approach - 2


DSData=compiled_data.loc[(((compiled_data['particle_isDown_noVelo']==True) | (compiled_data['particle_isLong_andUT']==True) ) & (compiled_data['particle_isElectron']==False) ) | (compiled_data['Ghost']==False)]


In [10]:
DSData=compiled_data.loc[((compiled_data['particle_isLong_andUT']==True) | (compiled_data['particle_isDown_noVelo']==True))
                        & (compiled_data['particle_isElectron']==False)]

In [11]:
GhostData=compiled_data.loc[compiled_data['Ghost']==True]

In [12]:

#save

file=uproot.recreate('./DataGenerated/02_Flagged.root')
file['Particle_Data']=DSData


file=uproot.recreate('./DataGenerated/02_Ghost.root')
file['Particle_Data']=GhostData